# Desafio para entrar no programa de aceleração Data Science:


Importar as bibliotecas necessarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

 Carregar o dataSet de train em partes para não comprometer a memoria ram do PC

In [ ]:
df = pd.read_csv(path,sep=',',encoding='latin',chunksize=1000000)
chunk_list = []  # list com partes do dataframe

# Loop para ir adicionando as colunas
for chunk in df:  
    
    # Filtrando as colunas do arquivo.
    chunk_list.append(chunk[['NU_INSCRICAO','NU_NOTA_MT','CO_UF_RESIDENCIA','SG_UF_RESIDENCIA','NU_IDADE','TP_SEXO','TP_COR_RACA','TP_NACIONALIDADE','TP_ST_CONCLUSAO','TP_ANO_CONCLUIU','TP_ESCOLA','TP_ENSINO','IN_TREINEIRO','TP_DEPENDENCIA_ADM_ESC','IN_BAIXA_VISAO','IN_CEGUEIRA','IN_SURDEZ','IN_DISLEXIA','IN_DISCALCULIA','IN_SABATISTA','IN_GESTANTE','IN_IDOSO','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','TP_LINGUA','TP_STATUS_REDACAO','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO','Q001','Q002','Q006','Q024','Q025','Q026','Q027','Q047']])
    
# Cria o DataFrame a partir da lista chunk_list
df_concat = pd.concat(chunk_list,ignore_index=True)

Elimina as linhas que contem colunas em branco:

In [ ]:
df_concat.dropna(subset=['NU_INSCRICAO','NU_NOTA_MT','CO_UF_RESIDENCIA','SG_UF_RESIDENCIA','NU_IDADE','TP_SEXO','TP_COR_RACA','TP_NACIONALIDADE','TP_ST_CONCLUSAO','TP_ANO_CONCLUIU','TP_ESCOLA','TP_ENSINO','IN_TREINEIRO','TP_DEPENDENCIA_ADM_ESC','IN_BAIXA_VISAO','IN_CEGUEIRA','IN_SURDEZ','IN_DISLEXIA','IN_DISCALCULIA','IN_SABATISTA','IN_GESTANTE','IN_IDOSO','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','TP_LINGUA','TP_STATUS_REDACAO','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_REDACAO','Q001','Q002','Q006','Q024','Q025','Q026','Q027','Q047'],inplace=True)


Separa os dados em features e labels:

In [ ]:
train_features=df_concat[['NU_NOTA_REDACAO','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC']]

train_labels=df_concat['NU_NOTA_MT']

Preparar os parametros do GridSearchCV para realizar cross validation para tuning dos hyperparameter:

In [ ]:
# Cria parametros para testes de random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Cria instancia do modelo
rf = RandomForestRegressor()
# Cria instancia do grid search
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

Inicia os testes para encontrar os melhores parametros do modelo

In [ ]:
grid_search.fit(train_features, train_labels)

Printa os melhore parametros:

In [ ]:
grid_search.best_params_

Seta os parametros baseados nas respostas:

In [ ]:
rf.n_estimators= 200
rf.min_samples_split= 12
rf.min_samples_leaf= 4
rf.max_features= 2
rf.max_depth=100
rf.bootstrap= True

Treina o modelo:

In [ ]:
rf.fit(train_features,train_labels)

Importa o dataframe de teste:

In [ ]:
df_test = pd.read_csv('test.csv')

Dropa as linhas com colunas em branco:

In [ ]:
df_test.dropna(subset=['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC'],inplace=True)
test_features=df_test[['NU_NOTA_REDACAO','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC']]

Faz a predict e salva o resultado em um dataframe de formato "NU_INSCRICAO,NU_NOTA_MT"

In [ ]:
resultado=rf.predict(test_features)

df_answer=df_test[['NU_INSCRICAO']]
df_answer.insert(1,'NU_NOTA_MT',resultado)

Salva o resultado no arquivo answer.csv

In [ ]:
df_answer.to_csv('answer.csv',index=False)